In [53]:
from bs4 import BeautifulSoup
from IPython.display import clear_output
from datetime import date, datetime, timedelta
from tqdm import tqdm
import urllib, urllib3
import requests, urllib.request, urllib.parse, csv, re, time, os, numpy as np, pandas as pd, json
key_minvv23 = '290dac7fcac7c640c069ced9e576bc00'

In [54]:
company_basicurl = 'http://kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyList.json?key={key}&companyNm={company}'
company_detailurl = 'http://kobis.or.kr/kobisopenapi/webservice/rest/company/searchCompanyInfo.json?key={key}&companyCd={companycode}'
warner_search = json.loads(urllib.request.urlopen(company_basicurl.format(key=key_minvv23, company='Warner')).read())
warner_codes = [each['companyCd'] for each in warner_search['companyListResult']['companyList'][:2]]

In [79]:
movie_codes = []

for each_companycode in warner_codes :
    warner_info = json.loads(urllib.request.urlopen(company_detailurl.format(key=key_minvv23, companycode=each_companycode)).read())
    for each in warner_info['companyInfoResult']['companyInfo']['filmos'] :
        movie_codes.append(int(each['movieCd']))

movie_codes = list(filter(lambda x : x>20100000, set(movie_codes)))

In [80]:
movie_info = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key={key}&movieCd={movie_code}'

In [81]:
wb_weblist = []
for each_code in movie_codes :
    movieapi = json.loads(urllib.request.urlopen(movie_info.format(key=key_minvv23, movie_code=each_code)).read())
    wb_weblist.append([movieapi['movieInfoResult']['movieInfo']['movieNmEn'], movieapi['movieInfoResult']['movieInfo']['movieCd']])

In [82]:
wb_csvlist = pd.read_csv('WB_Movies.csv')
wb_csvlist['theater_release'] = np.vectorize(lambda x : ''.join(re.findall(r'\d+', x)))(wb_csvlist['theater_release'].values)
wb_csvlist['vod_release'] = np.vectorize(lambda x : ''.join(re.findall(r'\d+', x)))(wb_csvlist['vod_release'].values)

In [87]:
json.loads(urllib.request.urlopen(movie_info.format(key=key_minvv23, movie_code=movie_codes[100])).read())

{'movieInfoResult': {'movieInfo': {'movieCd': '20125048',
   'movieNm': '호빗: 뜻밖의 여정',
   'movieNmEn': 'The Hobbit: An Unexpected Journey',
   'movieNmOg': '',
   'showTm': '169',
   'prdtYear': '2012',
   'openDt': '20121213',
   'prdtStatNm': '개봉',
   'typeNm': '장편',
   'nations': [{'nationNm': '미국'}, {'nationNm': '뉴질랜드'}],
   'genres': [{'genreNm': '어드벤처'}, {'genreNm': '판타지'}],
   'directors': [{'peopleNm': '피터 잭슨', 'peopleNmEn': 'Peter Jackson'}],
   'actors': [{'peopleNm': '마틴 프리먼',
     'peopleNmEn': 'Martin Freeman',
     'cast': '빌보 배긴스',
     'castEn': 'Bilbo Baggins'},
    {'peopleNm': '이안 맥켈런',
     'peopleNmEn': 'Ian McKellen',
     'cast': '간달프',
     'castEn': 'Gandalf'},
    {'peopleNm': '리차드 아미티지',
     'peopleNmEn': 'Richard Armitage',
     'cast': '',
     'castEn': ''},
    {'peopleNm': '케이트 블란쳇',
     'peopleNmEn': 'Cate Blanchett',
     'cast': '',
     'castEn': ''},
    {'peopleNm': '크리스토퍼 리',
     'peopleNmEn': 'Christopher Lee',
     'cast': '',
     'castEn': '

In [78]:
wb_csvlist

,name,admission,theater_release,vod_release
0,American Sniper,"345,782",20150114,20150416
1,Jupiter Ascending,"309,842",20150205,20150507
2,Run all night,"289,534",20150319,20150618
3,Mad Max: Fury Road,"3,884,129",20150514,20150813
4,San Andreas,"1,716,455",20150603,20150903
5,Gallows,"88,777",20150903,20151203
6,The Intern,"3,611,166",20150924,20151217
7,Pan,"186,174",20151008,20160107
8,Man From U.N.C.L.E,"108,552",20151028,20160128
9,In the Heart of the Sea,"809,841",20151203,20160303
